In [ ]:
import os
from selenium import webdriver
import requests
from selenium.webdriver.firefox.options import Options
import time
import re

In [ ]:
# judge if zhe folder exist. 判断图集的文件夹是否存在，如果不存在则建立一个
def fJudgeFile(vSeqName):
    global vSpomaFoldName
    # vSpomaFoldName = vOrfFolder + vSeqName + "\\SPOMA\\"
    vSpomaFoldName = "E:\\Store\\code\\02.Python\\03.Request\\Bio" + "\\" + vSeqName + "\\SPOMA二级结构分析\\"
    os.makedirs(vSpomaFoldName, exist_ok = True)
    os.chdir(vSpomaFoldName)
    os.makedirs("Pic", exist_ok = True)
    os.makedirs("Text", exist_ok = True)

In [ ]:
def fGetSopma(vSeq):
    options = Options()
    options.headless = True # 设置为无头浏览器
    vWebObj = webdriver.Firefox(options = options) # 创建浏览器
    # 设置url，发送请求
    url = 'https://npsa-prabi.ibcp.fr/cgi-bin/npsa_automat.pl?page=/NPSA/npsa_sopma.html'
    vWebObj.get(url)
    print("==========已打开网页==========")
    # 通过name属性找到输入框
    text = vWebObj.find_element_by_name('notice')
    # 递交序列    
    text.send_keys(vSeq)
    print("==========已输入序列==========")
    # 通过Xpath找到提交按钮
    vButton = vWebObj.find_element_by_xpath("//input[@value='SUBMIT']")
    # 点击按钮
    time.sleep(1)
    print("=========正在发送请求=========")
    vButton.click()
    print("=========开始等候分析=========")
    time.sleep(20)
    vHtml = vWebObj.page_source    
    #关闭浏览器
    vWebObj.quit()
    print("=========数据拉取结束=========")
    return(vHtml)

In [ ]:
def fSavePic(vHtml):
    global vCode
    vCode = re.search("/tmp/(.*?)\.sopma", vHtml, re.S).group(1)
    vPicA = "https://npsa-prabi.ibcp.fr/tmp/" + vCode + ".sopma_state.gif"
    vPicB = "https://npsa-prabi.ibcp.fr/tmp/" + vCode + ".sopma2.gif"
    vTxtA = "https://npsa-prabi.ibcp.fr/cgi-bin/npsa_mpsaview.pl?fn=/tmp/" + vCode + ".sopma&gui=antheprot"
    vTxtB = "https://npsa-prabi.ibcp.fr/tmp/" + vCode + ".sopma"
    vTxtC = "https://npsa-prabi.ibcp.fr/tmp/" + vCode + ".blastpsecpred"
    vTxtD =  "https://npsa-prabi.ibcp.fr/tmp/" + vCode + ".cluoutsecpred"
    with open("Pic\\sopma_state.gif", "wb") as vPic1:
        r = requests.get(vPicA)
        vPic1.write(r.content)
    with open("Pic\\sopma2.gif", "wb") as vPic2:
        r = requests.get(vPicB)
        vPic2.write(r.content)
    with open("Text\\spoma1.txt", "wb") as vTxt1:
        r = requests.get(vTxtA)
        vTxt1.write(r.content)
    with open("Text\\spoma2.txt", "wb") as vTxt2:
        r = requests.get(vTxtB)
        vTxt2.write(r.content)
    with open("Text\\blastP.txt", "wb") as vTxt3:
        r = requests.get(vTxtC)
        vTxt3.write(r.content)
    with open("Text\\clust.txt", "wb") as vTxt4:
        r = requests.get(vTxtD)
        vTxt4.write(r.content)         

In [ ]:
def fSoupSompa(vHtml):
    vSpomaText = "file:///" + vSpomaFoldName + "Text" + "\spoma1.txt"
    vSopma1 = "<B>SOMPA:</B><p>以下内容为来自<a target=\"_blank\" href=\"https://npsa-prabi.ibcp.fr/cgi-bin/npsa_automat.pl?page=/NPSA/npsa_sopma.html\">SOMPA</a>的二级结构预测。</p>" "<a href = " + vSpomaText + ">SPOMA分析文档</a>" + re.search("(\<pre\>[\s\S]*?)\<meta", vHtml, re.S).group(1)
    vSopma2 = vSopma1.replace("Sequence length", "序列长度（Sequence length）").replace("SOPMA", "SOPMA二级结构预测")
    vSopma3 = vSopma2.replace("Alpha helix", "α-螺旋（Alpha helix）   ").replace("3<sub>10 </sub> helix", "3<sub>10</sub>螺旋（3<sub>10 </sub> helix）   ")
    vSopma4 = vSopma3.replace("Pi helix", "Pi螺旋（Pi helix）    ").replace("Beta bridge", "β桥（Beta bridge）      ")
    vSopma5 = vSopma4.replace("Extended strand", "延伸链（Extended strand）    ").replace("Beta turn", "β转角（Beta turn）    ")
    vSopma6 = vSopma5.replace("Bend region", "Bend region              ").replace("Random coil", "无规则卷曲（Random coil）")
    vSopma7 = vSopma6.replace("Ambiguous states", "无法确定的部分               ").replace("Other states", "其它状态                  ")
    vSopma8 = vSopma7.replace(re.search("(\<pre\>Para[\s\S]*?\</pre\>)", vSopma7, re.S).group(1), "")
    vSopma9 = vSopma8.replace(re.search("(\<i\>\<b\>User[\s\S]*?\</i\>)", vSopma8, re.S).group(1), "")
    vSopma10 = vSopma9.replace("Prediction result file (text)", "用于预测的文件（txt）").replace("Intermediate result file (text)", "中间比对文件(txt)")
    vSopma11 = vSopma10.replace("/tmp/"+ vCode + ".sopma_state.gif", "file:///" + vSpomaFoldName + "\Pic" + "\sopma_state.gif")
    vSopma12 = vSopma11.replace("/tmp/"+ vCode + ".sopma2.gif", "file:///" + vSpomaFoldName + "\Pic" + "\sopma2.gif")
    vSopma13 = vSopma12.replace("/tmp/"+ vCode + ".sopma", "file:///" + vSpomaFoldName + "\Text" + "\spoma2.txt")
    vSopma14 = vSopma13.replace("/tmp/"+ vCode + ".blastpsecpred", "file:///" + vSpomaFoldName + "Text" + "\\blastP.txt")
    vSopma15 = vSopma14.replace("/tmp/"+ vCode + ".cluoutsecpred", "file:///" + vSpomaFoldName + "Text" + "\clust.txt")
    vSopma = vSopma15
    with open("sopma.html", "w") as f:
        f.write(vSopma)

In [ ]:
def main(vSeqName, vSeq):
    fJudgeFile(vSeqName)
    print("========已建立存储路径========")
    print("存储路径为：" + vSpomaFoldName)
    print("=========开始打开网页=========")
    vHtml = fGetSopma(vSeq)
    print("=======已获取SOMPA的分析======")
    fSavePic(vHtml)
    print("=======已存储图片及数据=======")
    fSoupSompa(vHtml)
    print("========已生成报告文档========")

In [ ]:
with open("name.txt", "r") as f:
    vSeqName = f.read()
with open("orf.txt", "r") as f:
    vSeq = f.read()
main(vSeqName, vSeq)